In [2]:
import numpy as np
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_1D import *
from ids import *
import itertools
%matplotlib inline

# Test on Block Rect Maxvol

In [68]:
from block_rect_maxvol import *

In [69]:
print list(itertools.combinations(xrange(5), 3))

[(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 2, 3), (0, 2, 4), (0, 3, 4), (1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4)]


In [76]:
def AllCombsDet(A, l, num_take_blocks=None, debug=False):
    # INPUT 
    # A -- matrix
    # l -- number of dimensions (derivatives), size of block is l+1
    # num_take_bocks -- blocks returnd by algorithm
    l1 = l + 1
    b_one = np.arange(l1)
    num_of_blocks = A.shape[0] // l1
    max_det = 0.0
    good_idx = None
    if num_take_blocks is None:
        num_take_blocks = A.shape[1] // l1 # Take square mat and test square block maxvol
    for i in itertools.combinations(xrange(num_of_blocks), num_take_blocks):
        # print i
        idx = np.hstack((b_one + j*l1 for j in i))
        Al = A[idx]
        B  = Al.dot(Al.T)
        cur_det = np.abs(la.det(B))
        if cur_det > max_det:
            max_det = cur_det
            good_idx = i
        if debug:
            print i, idx, cur_det
            
    return max_det, good_idx
    
    
def GetNumOfBlocks(idx, l):
    l1 = l+1
    b_one = np.arange(l1)
    idx_bl = idx[np.arange(0, len(idx), l1)] // (l1)
    assert( np.all(np.hstack((b_one + j*l1 for j in idx_bl)) == idx)  )
    return idx_bl
    
    

In [91]:
l = 2
num_of_blocks = 7
num_take_blocks = 4
n = num_of_blocks*(l+1)
m = (num_take_blocks - 2)*(l+1)

A = np.random.randn(n,m)
print AllCombsDet(A, l, num_take_blocks=num_take_blocks, debug=False)
idx_maxvol = rect_block_maxvol(A, l, Kmax = num_take_blocks*(l+1), 
                               max_iters=100, rect_tol = 0.05, tol = 0.0, debug = False, ext_debug = False)
idx_maxvol = idx_maxvol[:num_take_blocks*(l+1)]
idx_maxvol.sort()
A_mv = A[idx_maxvol]
idx_bl = GetNumOfBlocks(idx_maxvol, l)
print np.abs(la.det(A_mv.dot(A_mv.T))), idx_bl

(5.6466283188149548e-85, (0, 2, 3, 4))
2
4
5.64662831881e-85 [0 2 3 4]


# Old Tests

In [7]:
from numba import jit
# @jit()
def BF():
    global mat
    fnd = 0
    for i in xrange(1000000):
        # print i
        max_val = 20
        n, m = 10, 6
        mat_int = np.random.uniform( low=-max_val, high=max_val, size=n*m).reshape((n,m)).astype(int)
        mat = mat_int.astype(float)
        try:
            _,_,_,_, info = pluq_ids(mat, debug=False)
        except:
            np.savetxt('except.txt', mat_int)
            print 'except'
            return mat
        if info[1] > 100:
            np.savetxt('contra_{}_{}.txt'.format(fnd, info[1]), mat_int)
            return mat
            fnd += 1
            if fnd > 10:
                return
        

In [10]:
np.seterr(all='raise')
BF()

except


array([[ -5.,  14., -17., -18.,   4.,  19.],
       [ -1.,   0.,  16.,  -9.,   3., -14.],
       [ 12.,   0., -16.,  -4.,  -6.,  18.],
       [ -9.,  -4., -17.,  12.,  14.,   0.],
       [  0.,   2.,   0.,   0., -15.,   0.],
       [ 13.,  -5.,  -8.,  -7.,  13., -16.],
       [ -5.,   0.,   3.,   7.,  -7.,   0.],
       [ 19.,  -1., -10.,  18.,  17.,  -9.],
       [-15.,   6.,  -5., -12.,   2.,   4.],
       [-15., -19., -13., -14.,   0.,  -1.]])

In [13]:
mat.astype(int)

array([[ -5,  14, -17, -18,   4,  19],
       [ -1,   0,  16,  -9,   3, -14],
       [ 12,   0, -16,  -4,  -6,  18],
       [ -9,  -4, -17,  12,  14,   0],
       [  0,   2,   0,   0, -15,   0],
       [ 13,  -5,  -8,  -7,  13, -16],
       [ -5,   0,   3,   7,  -7,   0],
       [ 19,  -1, -10,  18,  17,  -9],
       [-15,   6,  -5, -12,   2,   4],
       [-15, -19, -13, -14,   0,  -1]])

In [16]:
mat_pull_2 = np.array([[ -5,  14, -17, -18,   4,  19],
       [ -1,   0,  16,  -9,   3, -14],
       [ 12,   0, -16,  -4,  -6,  18],
       [ -9,  -4, -17,  12,  14,   0],
       [  0,   2,   0,   0, -15,   0],
       [ 13,  -5,  -8,  -7,  13, -16],
       [ -5,   0,   3,   7,  -7,   0],
       [ 19,  -1, -10,  18,  17,  -9],
       [-15,   6,  -5, -12,   2,   4],
       [-15, -19, -13, -14,   0,  -1]], dtype=float)

In [17]:
_,_,_,_, info = pluq_ids(mat_pull_2, debug=False)
print info

FloatingPointError: divide by zero encountered in double_scalars

In [11]:
def zero(a, b):
    a = np.arrray([a])
    b = np.arrray([b])
    return a[0]/b[0]

In [12]:
zero(2.3, 0.0)

AttributeError: 'module' object has no attribute 'arrray'